In [1]:
%pip install xgboost


  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 236.4 kB/s eta 0:10:33
   ---------------------------------------- 0.5/150.0 MB 236.4 kB/s eta 0:10:33
   ---------------------------------------- 0.5/150.0 MB 236

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score,
    RocCurveDisplay,
    precision_recall_curve,
    auc,
)
from sklearn.pipeline import Pipeline


In [4]:
import pandas as pd

data_path = "D:/Edunutshell2/Titanic-Dataset.csv"

titanic = pd.read_csv(data_path)

print(titanic.shape)
titanic.head()


(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True, errors='ignore')


In [6]:
# Numeric columns
numeric_cols = titanic.select_dtypes(include=['number']).columns.tolist()
if 'Survived' in numeric_cols:
    numeric_cols.remove('Survived')

for col in numeric_cols:
    titanic[col] = titanic[col].fillna(titanic[col].median())

# Categorical columns
categorical_cols = titanic.select_dtypes(include=['object']).columns.tolist()

for col in categorical_cols:
    titanic[col] = titanic[col].fillna(titanic[col].mode()[0])


In [7]:
titanic_encoded = pd.get_dummies(titanic, drop_first=True)


In [3]:
X = titanic_encoded.drop('Survived', axis=1)
y = titanic_encoded['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
print(X_train.columns.tolist())


NameError: name 'titanic_encoded' is not defined

In [9]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)

random_search_rf = RandomizedSearchCV(
    rf,
    param_distributions=param_grid_rf,
    n_iter=10,
    scoring='roc_auc',
    cv=5,
    random_state=42,
    n_jobs=-1
)

random_search_rf.fit(X_train, y_train)

print("Best RF Params:", random_search_rf.best_params_)
rf_best = random_search_rf.best_estimator_


Best RF Params: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 5}


In [10]:
pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42))
])

param_grid_svm = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

grid_search_svm = GridSearchCV(
    pipe_svm,
    param_grid=param_grid_svm,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1
)

grid_search_svm.fit(X_train, y_train)

print("Best SVM Params:", grid_search_svm.best_params_)
svm_best = grid_search_svm.best_estimator_


Best SVM Params: {'svc__C': 1, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}


In [11]:
import joblib
joblib.dump(random_search_rf.best_estimator_,"model.pkl")

['model.pkl']

In [1]:
print(X_train.columns.tolist())


NameError: name 'X_train' is not defined